In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

Загрузка датасета

In [2]:
fields = pd.read_csv('Dataset/fields_anomaly.csv')
fields.head()

,field,region,land_type,fertility_score,season,crop,crop_predecessor,crop_yearold,area,hybrid,...,area_y,yield_fact,is_anomaly_p2o5,is_anomaly_k2o,is_anomaly_ph,is_anomaly_humus,is_anomaly_yield_plan,is_anomaly_yield_fact,is_anomaly_area,count_anomaly
0,22.47.01.001.01,Алтай,пашня,NaN,2023,Оз.пшеница мягкая,Пар; Яр.пшеница мягкая,NaN,306.22,Скипетр,...,306.22,2.465597,False,False,False,False,False,False,True,0
1,22.47.01.001.02,Алтай,пашня,NaN,2023,Пар,Пар; Яр.пшеница мягкая,NaN,63.16,NaN,...,NaN,NaN,False,False,False,False,True,True,False,1
2,22.47.01.002.00,Алтай,пашня,4.0,2023,Пар,Яр.пшеница мягкая,NaN,485.79,NaN,...,NaN,NaN,False,False,False,False,True,True,True,1
3,22.47.01.003.00,Алтай,пашня,4.0,2023,Оз.пшеница мягкая,Пар,NaN,452.25,Скипетр,...,452.25,2.602692,False,False,False,False,False,False,True,0
4,22.47.01.004.00,Алтай,пашня,3.0,2023,Оз.пшеница мягкая,Пар,NaN,269.64,Скипетр,...,269.64,2.836971,False,False,False,False,False,False,True,0


Нормализация категорий культур

In [3]:
s = """
Пар -
Оз.пшеница мягкая - озимая пшеница - пшеница - злаки
Залежь, лес -
Яр.пшеница мягкая - яровая пшеница - пшеница - злаки
Залежь -
Залежь, кустарник -
Люцерна 1 г.ж.корм - люцерна - бобовые
Кукуруза силос - кукуруза - злаки
Люцерна пр.лет корм - люцерна - бобовые
Одн.тр.(вика+овёс)корм - смесь вики и овса - бобовые и злаки (вот тут проблема, ведь вика - бобовые, а овес - злаки. Да и вообще непонятно, как учитывать однолетние / многолетние травы, ведь они часто представляют из себя смесь разных культур, поэтому, наверное, их лучше удалить или выделить отдельный класс) - травы
Строительство -
Яр.ячмень - ячмень - злаки
Чечевица - чечевица - бобовые
Соя - соя - бобовые
Люпин белый - люпин - бобовые
Яр.пшеница твердая - яровая пшеница - пшеница - злаки
Кукуруза зерно - кукуруза - злаки
Горох - горох - бобовые
Овес - овес - злаки
Не используется, чужая земля -
Гречиха - гречиха - гречишные
Сортоучасток -
Сах.свекла - свекла - амарантовые
Лён масличный - лен - льновые
Нут - нут - бобовые
Не занято -
Люцерна 1 г.ж.сем. - люцерна - бобовые
Эспарцет пр.лет сем. - эспарцет - бобовые
Не используется -
Одн.тр.райграс одн.с подс.(кл.кр.+тим.корм)корм - райграс - злаки
Кл.кр.+тим.1 г.ж.корм - травы
Кл.кр.+тим.пр.лет корм - травы
Одн.тр.(вика+овёс)с подс.(кл.кр.+тим.корм)корм - смесь вики и овса - бобовые и злаки
Одн.тр.(вика+овёс)с подс.(кл.кр.+тим.корм)сем. - смесь вики и овса - бобовые и злаки
Мн.тр.разн.бот.сост.пр.лет корм - травы
Оз.рапс - озимый рапс - рапс - капустные
Козлятник пр.лет корм - козлятник - бобовые
Одн.тр.яр.ячмень корм - яровой ячмень - ячмень - злаки
Одн.травы(с подс.мн.трав) - травы
Одн.тр.(смеси з.и з.б.)корм - травы
Луг -
Клевер кр.пр.лет корм - клевер - бобовые
Одн.тр.оз.рожь корм - озимая рожь - рожь - злаки
Оз.рожь - озимая рожь - злаки
Кормовые бобы - бобы - бобовые
Оз.тритикале - озимая тритикале - тритикале - злаки
Подсолнечник - подсолнечник - астровые
Подсолнечник кондит. - подсолнечник - астровые
Яр.рапс - яровой рапс - рапс - капустные
Одн.тр.овёс корм - овес - злаки
Кукуруза корнаж - кукуруза - злаки
Площадка под корма -
Пастбище естественное -
Аренда -
Одн.тр.оз.пшеница мягк.корм - озимая пшеница - пшеница - злаки
Тимофеевка пр.лет корм - тимофеевка - злаки
Мн.травосм.пр.лет корм - травы
Смесь зерн.и зернобоб.с подс.(мн.травосм.корм)фураж - смесь зерновых и зернобобовых - бобовые и злаки
Пастбище на пашне -
Мн.травосм.1 г.ж.корм - травы
Одн.тр.(смеси з.и з.б.)с подс.(кл.кр.+тим.корм)корм - травы
Овёс с подс.(пастб.травосмесь) - овес - злаки
Одн.тр.суданская тр.корм. - травы
Эспарцет 1 г.ж.корм - эспарцет - бобовые
Эспарцет пр.лет корм - эспарцет - бобовые
Люцерна пр.лет сем. - люцерна - бобовые
Оз.рыжик - озимый рыжик - рыжик - капустные
Загон -
Сорго зерновое - сорго - злаки
Эсп.+костр.пр.лет корм - смесь бобовых и злаков - бобовые и злаки
Яр.ячмень с подс.(мн.травосм.корм) - яровой ячмень - ячмень - злаки
Одн.тр.овёс с подс.(мн.травосм.корм)корм - овес - злаки
Люц.+кострец пр.лет корм - могар - злаки
Одн.тр.могар корм,подст. - могар - злаки
Смесь зерн.фураж - злаки
Одн.тр.суданская тр.семена - травы
Одн.тр.озимая тритикале корм - озимая тритикале - тритикале - злаки
Эсп.+костр.1 г.ж.корм - смесь эспарцета и костреца - бобовые и злаки
Райграс пастб.пр.лет сем. - райграс - злаки
Яр.пшеница мягк.с подс.(кл.кр.сем.) - яровая пшеница - пшеница - злаки
Ежа пр.лет сем. - ежа - злаки
Клевер кр.пр.лет сем. - клевер - бобовые
Клевер бел.пр.лет сем. - клевер - бобовые
Яр.пшеница мягк.с подс.(кл.кр.+тим.корм) - яровая пшеница - пшеница - злаки
Овёс с подс.(кл.кр.сем.) - овес - злаки
Тимофеевка пр.лет сем. - тимофеевка - злаки
Фестулолиум пр.лет сем. - фестулолиум - злаки
Одн.тр.(вика+овёс)с подс.(кл.кр.сем)семена - травы
Фестулолиум пр.лет корм - фестулолиум - злаки
Райграс пастб.1 г.ж.сем. - райграс - злаки
Горох с под(клевер белый сем) - горох - бобовые
Кл.кр.+фестул.пр.лет корм - травы
Кл.кр.+райгр.пастб.пр.лет корм - травы
Эсп.+житн.пр.лет корм - травы
Эспарцет 1 г.ж.сем. - эспарцет - бобовые
Одн.тр.фацелия семена - фацелия - водолистниковые
Одн.тр(оз.тритикале+оз.вика) корм - смесь тритикале и вики - бобовые и злаки
Одн.тр(оз.тритикале+оз.вика) семена - смесь тритикале и вики - бобовые и злаки
Мн.травы рожь многолетняя 1 г.ж корм - рожь - злаки
Одн.тр.(вика+овёс)сем. - смесь овса и вики - злаки
Подсолнечник (аренда) - подсолнечник - астровые
Мн.травы (эспарцет+люцерна) 1г.ж. сем. - смесь эспарцета и люцерны - бобовые
Чечевица красная - чечевица - бобовые
Чечевица зеленая - чечевица - бобовые
Кукуруза гибрид - кукуруза - злаки
Одн.тр.(оз.пшеница+оз.вика)корм - смесь пшеницы и вики - бобовые и злаки
Животноводческий комплекс -
Овёс с подс.(кл.кр+тим.корм) - овес - злаки
Одн.тр.сорго-суданский гибрид корм - смесь сорго и суданской травы - злаки
Козлятник 1 г.ж.корм - козлятник - бобовые
Горчица сарептская - горчица - капустные
Одн.тр.сорго зерновое корм - сорго  - злаки
Смесь зерн.и зернобоб.фураж - смесь зерновых и зернобобовых - бобовые и злаки
Козлятник 1 г.ж семена - козлятник - бобовые
Одн.тр.(оз.рожь+оз.вика)корм - смесь ржи и вики - бобовые и злаки
Одн.тр.чумиза корм - чумиза - злаки
Одн.тр.сорго-суданский гибрид корм - злаки
Клевер бел.1 г.ж.сем. - клевер - бобовые
Ежа 1 г.ж.сем. - ежа - злаки
Клевер кр.1 г.ж.сем. - клевер - бобовые
Тимофеевка 1 г.ж.сем. - тимофеевка - злаки
Фестулолиум 1 г.ж.сем. - фестулолиум - злаки
Подсолнечник гибридизация - подсолнечник - астровые
Эсп.+костр.пр.лет сем. - смесь эспарцета и костреца - бобовые и злаки
""".replace("ё", "е")

In [4]:
s = s.split(sep='\n')

In [5]:
for i in range(len(s)):
    s[i] = s[i][:s[i].find('-')] + s[i][s[i].rfind("-"):]

In [6]:
rename = dict()
for i in s:
    ind = i.find("-")
    if i[:ind - 1] not in rename.keys():
        rename[i[:ind - 1]] = " ".join(i[ind + 2:].split())

In [7]:
rename['Одн.тр.сорго-суданский гибрид корм'] = "злаки"
fields["crop"] = fields["crop"].apply(lambda x: rename[x.replace("ё", "е")])

In [8]:
fields = fields.reset_index(drop=True)

Группировка регионов

In [9]:
# region_name = {
#     'Воронеж': 'КурВо',
#     'Калуга': 'МосКаРя',
#     'Рязань': 'МосКаРя',
#     'Курск': 'КурВо',
#     'Татарстан': "ПФО",
#     'Новосибирск': "СФО",
#     'Башкирия': "ПФО",
#     'Оренбург': "ПФО",
#     'Москва': 'МосКаРя',
#     'Самара': "ПФО",
#     'Тюмень': "СФО",
#     'Алтай': "СФО",
#     'СПб': 'СЗФО',
# }

region_name = {
    'Воронеж': 'КурВо',
    'Курск': 'КурВо',

    'Калуга': 'МосКаРя',
    'Рязань': 'МосКаРя',
    'Москва': 'МосКаРя',

    'Татарстан': "Другое",
    'Новосибирск': "Другое",
    'Башкирия': "Другое",
    'Оренбург': "Другое",
    'Самара': "Другое",
    'Тюмень': "Другое",
    'Алтай': "Другое",
    'СПб': 'Другое',
}

In [10]:
fields['region_raw'] = fields['region']
fields["region"] = fields["region"].apply(lambda x: region_name[x])

In [11]:
fields.head()

,field,region,land_type,fertility_score,season,crop,crop_predecessor,crop_yearold,area,hybrid,...,yield_fact,is_anomaly_p2o5,is_anomaly_k2o,is_anomaly_ph,is_anomaly_humus,is_anomaly_yield_plan,is_anomaly_yield_fact,is_anomaly_area,count_anomaly,region_raw
0,22.47.01.001.01,Другое,пашня,NaN,2023,злаки,Пар; Яр.пшеница мягкая,NaN,306.22,Скипетр,...,2.465597,False,False,False,False,False,False,True,0,Алтай
1,22.47.01.001.02,Другое,пашня,NaN,2023,,Пар; Яр.пшеница мягкая,NaN,63.16,NaN,...,NaN,False,False,False,False,True,True,False,1,Алтай
2,22.47.01.002.00,Другое,пашня,4.0,2023,,Яр.пшеница мягкая,NaN,485.79,NaN,...,NaN,False,False,False,False,True,True,True,1,Алтай
3,22.47.01.003.00,Другое,пашня,4.0,2023,злаки,Пар,NaN,452.25,Скипетр,...,2.602692,False,False,False,False,False,False,True,0,Алтай
4,22.47.01.004.00,Другое,пашня,3.0,2023,злаки,Пар,NaN,269.64,Скипетр,...,2.836971,False,False,False,False,False,False,True,0,Алтай


Сохранение датасета

In [12]:
fields.to_csv("Dataset/fields_norm.csv", index=False)